# 목표 : 비슷한 아티스트를 가진 플레이리스트들을 그룹화해보자

---

## 라이브러리, 데이터 불러오기

In [10]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

In [20]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

In [21]:
# 데이터 확인
display(train_df.head(1))
display(song_df.head(1))
type(train_df), type(song_df)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0


(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [13]:
genre_gn_all = pd.read_json('data/genre_gn_all.json', typ = 'series')

In [14]:
pd.DataFrame(genre_gn_all).T

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,GN2802,GN2803,GN2804,GN2805,GN2806,GN2900,GN2901,GN2902,GN2903,GN3000
0,발라드,세부장르전체,'80,'90,'00,'10-,댄스,세부장르전체,'80,'90,...,ASMR/자연,힐링/명상/요가,집중력,숙면유도,반려동물,뮤지컬,세부장르전체,국내뮤지컬,국외뮤지컬,크리스마스


---

## 사용할 컬럼 추리기
- 내가 사용할 컬럼은 train_df의 `id`, `plylst_title`, `songs` 와  song_df의 `song_id`, `artist_name_basket`
- `songs`와 `song_id`를 기준으로 song_df의 `artist_name_basket`를 train_df에 조인시키기 위한 사전작업임.

In [15]:
train_df.drop(['tags','like_cnt','updt_date'], axis=1, inplace=True)

In [16]:
train_df.head()

,id,plylst_title,songs
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954..."
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427..."
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598..."
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033..."
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [7]:
# train_df['songs']의 값인 리스트 속 int를 str로 바꿔주는 함수.
# 사용하진 않아서 주석처리

# train_df['songs_str'] = train_df['songs'].apply(lambda x : list(map(str, x)))
# train_df.drop(['songs'], axis=1, inplace=True)
train_df.head(1)

In [23]:
song_df.drop(['song_gn_dtl_gnr_basket','issue_date','album_name','album_id','song_gn_gnr_basket'],axis=1,inplace=True)
song_df['song_id'] = song_df['id']
song_df.drop('id',axis=1,inplace=True)
song_df.head(1)

,artist_id_basket,song_name,artist_name_basket,song_id
0,[2727],Feelings,[Various Artists],0


## `Artist`를 `songs`에 매핑시키기
- song_meta에 `artist_name_basket`가 있고 train_df에 `song_id`가 있음   
- song_meta와 train데이터를 조인 시킬 때 `songs`컬럼과 매칭될 수 있도록 `artist_name_basket`을 리스트 형식으로 조인시킴.      
- 이 결과 각 곡에 아티스트를 매핑하는 것보다 데이터 프레임이 방대해지는 것을 방지할 수 있었음.  
 

In [10]:
display(train_df.head(1))
display(song_df.head(1))

,id,plylst_title,songs
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954..."


,artist_id_basket,song_name,artist_name_basket,song_id
0,[2727],Feelings,[Various Artists],0


In [14]:
def match_artist(x):
    temp_list = []
    for song_ids in x:
#         print(song_dict[song_ids]['artist_name_basket'])
        temp_list.extend(song_dict[song_ids]['artist_name_basket'])

    temp_list = list(set(temp_list))
    return temp_list

train_df['artist_list'] = train_df['songs'].map(match_artist)

In [15]:
train_df.head(1)

,id,plylst_title,songs,artist_list
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[Lily Of The Valley, Red Hot Chili Peppers, Co..."
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[타루, 임주연, 제이레빗(J Rabbit), 라이너스의 담요, 박명수, 이상순, ..."
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[달 좋은 밤, 소울사이어티, 참깨와 솜사탕, PERC%NT, 슈가볼, 김박첼라, ..."
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[장희원, Ku Il Oh Trio, 이석훈, Katie Lopez, Ashley ..."
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...","[유피, 핑클 (Fin.K.L), 지누션, 샵, 마로니에, 원타임, 터보, 보아 (..."
...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[Carcass, DevilDriver, Slipknot, Deicide, Arch..."
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...","[Flatland Funk, Komes, Bsharry, Ste Ingham, Sp..."
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[씨야, 가비엔제이, 윤종신, 김창기, 벤, 김진호 (SG워너비), 양희은, 나비,..."
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[Angus & Julia Stone, Ben Howard, BØRNS, The S..."


In [26]:
train_df[['plylst_title', 'artist_list']]

,plylst_title,artist_list
0,여행같은 음악,"[Lily Of The Valley, Red Hot Chili Peppers, Co..."
1,요즘 너 말야,"[타루, 임주연, 제이레빗(J Rabbit), 라이너스의 담요, 박명수, 이상순, ..."
2,"편하게, 잔잔하게 들을 수 있는 곡.-","[달 좋은 밤, 소울사이어티, 참깨와 솜사탕, PERC%NT, 슈가볼, 김박첼라, ..."
3,크리스마스 분위기에 흠뻑 취하고 싶을때,"[장희원, Ku Il Oh Trio, 이석훈, Katie Lopez, Ashley ..."
4,추억의 노래 ㅋ,"[유피, 핑클 (Fin.K.L), 지누션, 샵, 마로니에, 원타임, 터보, 보아 (..."
...,...,...
115066,METAL E'SM #2,"[Carcass, DevilDriver, Slipknot, Deicide, Arch..."
115067,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[Flatland Funk, Komes, Bsharry, Ste Ingham, Sp..."
115068,#1. 눈물이 앞을 가리는 나의_이야기,"[씨야, 가비엔제이, 윤종신, 김창기, 벤, 김진호 (SG워너비), 양희은, 나비,..."
115069,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[Angus & Julia Stone, Ben Howard, BØRNS, The S..."


In [30]:
train_df['artist_cnt'] = train_df['artist_list'].map(lambda x : len(x))
train_df.head(1)

,id,plylst_title,songs,artist_list,artist_cnt
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[Lily Of The Valley, Red Hot Chili Peppers, Co...",17


In [36]:
def list_merger(x) :
    temp_list.extend(x)

temp_list = []
train_df['artist_list'].map(list_merger)
unique_artists = list(set(temp_list))
num_artists = len(unique_artists)

In [53]:
print(num_artists)
print(sorted(unique_artists))

99656
['', ' A.R.D.I.', ' Al Kasha', ' Benny Benassi', ' Black Pata', ' Budapest Failoni Chamber Orchestra', ' Chloe Martin', ' Coa White', ' Coro Del Teatro Alla Scala, Milano', ' Curbi', ' DJ Deeon', ' DJ True', ' David Lloyd-Jones', ' Dbrown', ' English Northern Philharmonia', ' Frank Tate', ' Hungarian State Opera Orchestra', ' Irish Film Orchestra', ' Lazy Hammock', ' Leonard Bernstein', ' MC Meta', ' Moon Rocket', ' Mugwanti', ' New Zealand Symphony Orchestra', ' Penn Masala', ' Prague Chamber Orchestra', ' ROSTAM', ' Raven & Kreyn', ' Richard Earnshaw', ' Robert Koenig', ' Roman Polonsky', ' Rostam', ' Scotty', ' Sigridur Nielsdottir', ' Simon Watkin', ' SkyKeeper', ' Swonderful Orchestra', ' Tara McDonald', ' Teairra Marie', ' The Bassalindos', ' The Rare Birds', ' The Rude Boy Of House', ' The Subs', ' Tiff Lacey', ' Ulster Orchestra', ' Vengeance', ' Wiener Singakademie', ' William Butt', ' William Naraine', ' Ying Yan Choir', ' monyamonya', ' 김민지', ' 임우(林雨)', '!!!', '!DelaDa

In [46]:
artist_to_id = {}

id_to_artist = {}

for i,v in enumerate(unique_artists):
    id_to_artist[i] = v
    
artist_to_id = {v:k for k,v in id_to_artist.items()}

def artist_id_trans(x):
    temp_list = []
    for artist in x : 
        temp_list.append(artist_to_id[artist])
    
    return temp_list
train_df['artist_id'] = train_df['artist_list'].map(artist_id_trans)

In [55]:
train_df.head(1)

,id,plylst_title,songs,artist_list,artist_cnt,artist_id
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[Lily Of The Valley, Red Hot Chili Peppers, Co...",17,"[51388, 87460, 71903, 78840, 86864, 68964, 971..."
